In [1]:
'''
Author: Ivan Gvozdanovic
Date: 10/19/2023

Finding optimal tour in Traveling Salesman Problem, using tabular Q-learning algorithm.

''';

In [2]:
'''
fix the connectivity of the graph if we are picking multiple edges 

''';

In [3]:
import scipy
import scipy.io
from datetime import date, time, datetime as Date, time, datetime
from scipy import optimize
import networkx as nx
import ast
import random
import os
import re
import numpy as np
import time as Time
from operator import itemgetter
import math as m
import copy as cpy
import matplotlib.pyplot as plt
import os
import itertools
import pickle

from TabularTspMultiEdgeSwapENV import PolytopeENV as Env



$\Large \textbf{Q-learning algorithm}$

In [4]:
from Q_learning import Q_learning, EdgeSwap_Q_learning, Multi_EdgeSwap_Q_learning

$\Large \textbf{Extracting the optimal policy}$

In [5]:
from optimal_policy_extraction import optimal_policy_time_independent, \
                                      edge_swap_policy_evaluation, \
                                      average_optimal_policy, \
                                      compute_path_cost, \
                                      compute_state_radius

$\Large \textbf{Reward functions}$

In [6]:
from reward_functions import reward_cost, calculate_reward1, calculate_reward2, calculate_reward3

$\Large \textbf{Initial Solution}$

In [7]:
from helper_functions import create_tsp_polytope_graph, extract_distance_matrix, create_state_graph, create_state_edges


objective_table = [] # cost vector
initial_states = {} # dictionary holding the initial states.
reward_lists = [] # list holding the rewrds for each edge in the subproblem. We read it off to compute the travel cost.

patches = 1
nodes_per_patch = 8

#Pick the file to the problem:
file = 'TSP_MultiDiscrete_DQN'


available_actions, initial_states, distance_matrix, objective_table, reward_list = create_tsp_polytope_graph(nodes_per_patch, 
                                                                                                             patches, 
                                                                                                             initial_states, 
                                                                                                             reward_lists,
                                                                                                             file,
                                                                                                             reward_cost,
                                                                                                             1,200)




print(len(initial_states[0]),initial_states[0])
print(available_actions)

Path nodes: 
 [0, 2, 7, 3, 1, 4, 5, 6]
Path edges: 
 [(0, 2), (2, 7), (7, 3), (3, 1), (1, 4), (4, 5), (5, 6), (6, 0)]
sub nodes: 
 [0, 2, 7, 3, 1, 4, 5, 6]
sub sorted nodes: 
 [0, 1, 2, 3, 4, 5, 6, 7]
sub edges: 
 [(0, 2), (0, 6), (2, 7), (7, 3), (3, 1), (1, 4), (4, 5), (5, 6)]
Initial solution: 
 28 [0 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 1 0 0]
is connected:  True
Is graph connected:  True
Edges for reward list:  [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 4), (3, 5), (3, 6), (3, 7), (4, 5), (4, 6), (4, 7), (5, 6), (5, 7), (6, 7)]
Sub reward list: 
 28 [169, 83, 141, 40, 27, 145, 179, 15, 137, 60, 86, 71, 181, 117, 133, 190, 5, 171, 23, 159, 184, 48, 164, 19, 142, 37, 70, 45]
Initial reward:  -845
Margin: 
 [2. 2. 2. 2. 2. 2. 2. 2.]
Number of actions is 20
28 [0 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 1 0 0]
[array([ 0,  1, -1,  0,  0,  0,  0, -1,  1,  0,  0,  0

In [8]:
from exact_solution import TSP_integer_program

tour_edges = TSP_integer_program(distance_matrix) # solve TSP-MZT integer program.
print("Tour edges: \n", tour_edges)

tour_edges = [(min(e),max(e)) for e in tour_edges]
or_reward = 0
for e in tour_edges:
    or_reward += objective_table[e]
print("Reward: ", or_reward)

Status: Optimal
Optimal route: 0 -> 5  1 -> 2  2 -> 6  3 -> 4  4 -> 0  5 -> 1  6 -> 7  7 -> 3  
Total Distance: 289.0
Tour edges: 
 [(0, 5), (1, 2), (2, 6), (3, 4), (4, 0), (5, 1), (6, 7), (7, 3)]
Reward:  289


In [9]:
from policy_functions import multi_edge_swap_policy

In [10]:
state = [0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1]
state_edges = create_state_edges(8, state)
action = multi_edge_swap_policy(None, state, state_edges, 1, 3)
print(action)

[4, 6, 7]


$\Large \textbf{Main training loop}$

In [11]:
save_data = True  # save Q table data and cost vector data.
save_plots = False  # save the plots


# Model Parameters
epsilon = 1  # exploration parameter.
reward_parameter = 1
lr = 0.07  # learning rate.
discount_factor = 0.9  # discount parameter for the reward.
episode_numbers = [1000]  # number of episodes we run the algorithm on.
path_numbers = [20] # number of paths we run for each episode.
max_path_lengths = [100] # maximum number of steps allowed per a path.
table_size = available_actions[0].shape[0] # the size of each state and action vector.

k_swaps = 3
n_step_lookup = 10

# Set the correct Q-learning configuration.
episode_num = episode_numbers[0]
path_num = path_numbers[0]
show_path_num = path_num*50
max_path_length = max_path_lengths[0]


combinations = list(itertools.combinations(range(nodes_per_patch), 3))
# Convert each tuple to a list
action_space_values = [list(pair) for pair in combinations]
action_space_size = nodes_per_patch-1

lb = -2
ub = 3

best_states_size = 10
best_states = {0: (initial_states[0], reward_cost(reward_list, initial_states[0]))}
print(best_states)

print(action_space_values)

{0: (array([0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0], dtype=int32), -845)}
[[0, 1, 2], [0, 1, 3], [0, 1, 4], [0, 1, 5], [0, 1, 6], [0, 1, 7], [0, 2, 3], [0, 2, 4], [0, 2, 5], [0, 2, 6], [0, 2, 7], [0, 3, 4], [0, 3, 5], [0, 3, 6], [0, 3, 7], [0, 4, 5], [0, 4, 6], [0, 4, 7], [0, 5, 6], [0, 5, 7], [0, 6, 7], [1, 2, 3], [1, 2, 4], [1, 2, 5], [1, 2, 6], [1, 2, 7], [1, 3, 4], [1, 3, 5], [1, 3, 6], [1, 3, 7], [1, 4, 5], [1, 4, 6], [1, 4, 7], [1, 5, 6], [1, 5, 7], [1, 6, 7], [2, 3, 4], [2, 3, 5], [2, 3, 6], [2, 3, 7], [2, 4, 5], [2, 4, 6], [2, 4, 7], [2, 5, 6], [2, 5, 7], [2, 6, 7], [3, 4, 5], [3, 4, 6], [3, 4, 7], [3, 5, 6], [3, 5, 7], [3, 6, 7], [4, 5, 6], [4, 5, 7], [4, 6, 7], [5, 6, 7]]


In [12]:
# Convert dictionary values to a list of arrays
visited_states = [np.array(initial_states[0])]
visited_states = np.stack(visited_states)


#Initialize the environment.
env = Env(initial_states[0], # initial_state
         reward_list, # edge_weights
         episode_num, # total_episodes
         max_path_length,
         50, # show_path_num
         visited_states,  # visited_states
         available_actions, # basis_moves
         nodes_per_patch, # node_num
         0, # P
         best_states,
         best_states_size,
         objective_table,
         False,
         discount_factor,
         reward_function = reward_cost
         )

<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast


In [14]:
#Start the Q-learning algorithm.

start_time = Time.time()

Q = Multi_EdgeSwap_Q_learning(epsilon, 
                               episode_num, 
                               path_num, 
                               table_size, 
                               max_path_length, 
                               discount_factor, 
                               env, 
                               lr, 
                               save_plots, 
                               nodes_per_patch,
                               action_space_values,
                               action_space_size,
                               n_step_lookup,
                               k_swaps)



#Save the Q table.
if save_data:
    time = Time.localtime()
    current_time = Time.strftime("%H-%M-%S", time)
    date = datetime.now()
    d = date.isoformat()[0:10]
    data_save = [Q]
    data_save = np.array(data_save, dtype=object)
    print(d[0:10])
    np.save('Models/'
            +'Q_EP_'
            +str(episode_num)+'_P_'+str(path_num)
            +'_PL_'+ str(max_path_length)+'_Date_'+d+'_.npy',data_save)
    

    
end_time = Time.time()

print(f'It took {(end_time-start_time)/60} minutes to run {episode_num} episodes.')




 Episode Number:  1000 
 Path Number:  20 
 Learning Rates:  0.07 
 Gamma:  0.9 
 Table length:  28 
 Maximum Path Length:  100
##############################  Episode:  1   ##############################
[(0, 2), (0, 6), (1, 3), (1, 4), (2, 7), (3, 7), (4, 5), (5, 6)]
[1, 2, 4]
[(0, 6), (1, 3), (2, 7)]
[(0, 2), (0, 3), (1, 4), (1, 7), (2, 6), (3, 7), (4, 5), (5, 6)]
The BEST STATES DICT is full, and we found a better state, remove the worst state  10
[(0, 2), (0, 3), (1, 4), (1, 7), (2, 6), (3, 7), (4, 5), (5, 6)]
[0, 2, 5]
[(0, 2), (1, 4), (3, 7)]
[(0, 3), (0, 4), (1, 6), (1, 7), (2, 3), (2, 6), (4, 5), (5, 7)]
The BEST STATES DICT is full, and we found a better state, remove the worst state  10
[(0, 3), (0, 4), (1, 6), (1, 7), (2, 3), (2, 6), (4, 5), (5, 7)]
[1, 3, 5]
[(0, 4), (1, 7), (2, 6)]
[(0, 3), (0, 7), (1, 6), (1, 7), (2, 3), (2, 4), (4, 5), (5, 6)]
[(0, 3), (0, 7), (1, 6), (1, 7), (2, 3), (2, 4), (4, 5), (5, 6)]
[0, 1, 6]
[(0, 3), (0, 7), (4, 5)]
[(0, 5), (0, 7), (1, 6), (1

SystemExit: 

C:\Users\gvozd\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


$\Large \textbf{Examine the optimal policy}$

In [ ]:
data_load = np.load('Models/'
                    +'Q_EP_'
                    +str(episode_num)
                    +'_P_'+str(path_num)
                    +'_PL_'
                    + str(max_path_length)
                    +'_Date_2024-09-26'
                    +'_.npy',allow_pickle=True)
Q = data_load[0] 

with open('Models' + os.sep + 'visited_states.pkl', 'rb') as f:
    starting_states = pickle.load(f)
# print(starting_states, starting_states.shape)

# #Initialize the environment.
# env = Env(initial_states[0], # initial_state
#          reward_list, # edge_weights
#          episode_num, # total_episodes
#          show_path_num, # show_path_num
#          [initial_states[0]],  # visited_states
#          available_actions, # basis_moves
#          nodes_per_patch, # node_num
#          0, # P
#          lb, #lb
#          reward_function = reward_cost,
#          )


# path, final_state, maximum_pair = optimal_policy_time_independent(Q,
#                                                                 env,
#                                                                 reward_list,
#                                                                 table_size,
#                                                                 available_actions,
#                                                                 max_path_length,
#                                                                 lb,
#                                                                 ub,
#                                                                 save_plots)


# print("Path: \n", path)
# print("Max pair:", maximum_pair)




# Convert dictionary values to a list of arrays
# visited_states = [np.array(initial_states[0])]
# visited_states = np.stack(visited_states)
print(np.array([initial_states[0]]))
trajectory_num = 10
random_initial_states = starting_states[np.random.choice(starting_states.shape[0], trajectory_num, replace=False)]
random_initial_states = np.concatenate((np.array([initial_states[0]]), random_initial_states), axis=0)


for t in range(trajectory_num):
    print("\n")
    print("####################################################################################################")
    print("####################################################################################################")
    print("####################################################################################################")
    print("####################################################################################################")
    print("####################################################################################################")
    print("\n")
    print("Initial Solution: \n", random_initial_states[t,:])
    #Initialize the environment.
    env = Env(random_initial_states[t,:], # initial_state
             reward_list, # edge_weights
             episode_num, # total_episodes
             100,
             50, # show_path_num
             starting_states,  # visited_states
             available_actions, # basis_moves
             nodes_per_patch, # node_num
             0, # P
             best_states,
             best_states_size,
             objective_table,
             True,
             discount_factor,
             reward_function = reward_cost
             )

    edge_swap_policy_evaluation(Q,env,reward_list,20,action_space_values,action_space_size, nodes_per_patch)



